First convert Kinect skeleton to 

In [1]:
# hide
%load_ext autoreload
%autoreload 2

In [2]:
import bbsQt

In [7]:
import json
import numpy as np
from glob import glob 
import bbsQt.model.BBS_pp_utils as bbpp

import matplotlib.pyplot as plt 
import os

from bbsQt.model.data_preprocessing import save_npy

In [8]:
def kinect2mobile(karr, marr):
    """fills mobile_skeleton array with KINECT_BBS skeleton array
       with appropriate conversions.
       
       KINECT_BBS names are different from 
    """
    assert len(karr) == len(marr), "coco array and mobile array are not in the same length"
    
    K2M = {"SHOULDER_LEFT":"l_shoulder",
           "SHOULDER_RIGHT":"r_shoulder",
           "ELBOW_LEFT":"l_elbow", 
           "ELBOW_RIGHT":"r_elbow", 
           "WRIST_LEFT":"l_hand", 
           "WRIST_RIGHT":"r_hand",
           "HIP_LEFT":"l_hip", 
           "HIP_RIGHT":"r_hip", 
           "KNEE_LEFT":"l_knee", 
           "KNEE_RIGHT":"r_knee",
           "ANKLE_LEFT":"l_foot", 
           "ANKLE_RIGHT":"r_foot"}

    # Assume neck is the mid point of shoulders
    marr['xneck'] = (karr['xSHOULDER_LEFT'] + karr['xSHOULDER_RIGHT'])/2
    marr['yneck'] = (karr['ySHOULDER_LEFT'] + karr['ySHOULDER_RIGHT'])/2
    
    # Assume (tip of) head is 2 times nose from neck
    marr['xhead'] = karr['xNOSE']# + (karr['xNOSE']-karr['xNECK'])
    marr['yhead'] = karr['yNOSE']# + (karr['yNOSE']-karr['yNECK'])
    
    marr['xpelvis'] = (karr['xHIP_LEFT'] + karr['xHIP_RIGHT'])/2
    marr['ypelvis'] = (karr['yHIP_LEFT'] + karr['yHIP_RIGHT'])/2
    
    for common_field in K2M:
        for prefix in ['x','y']:
            marr[prefix+K2M[common_field]] = karr[prefix+common_field]
    
    marr['frame'] = karr['frame']

get_frame = lambda fn : int(fn.split("/")[-1].split(".")[0])

def add_frame_joint(arr, this_dict, iframe):
    for feature in this_dict:
        arr[iframe]['x'+feature] = this_dict[feature][0]
        arr[iframe]['y'+feature] = this_dict[feature][1] # [2] == depth
    
    
def get_skeletons(scene, ll, iframe):
    for ss, obj in zip(scene, ll['objects']):
        add_frame_joint(ss['data'], obj['keypoints'], iframe)
        ss['data'][iframe]['frame'] = iframe + 1

def load_scene(fn_dir):
    fn_frames = glob(fn_dir+"????.json")
    fn_frames.sort()
    nframe = len(fn_frames)
    
    if nframe == 0:
        print("No frames found", fn_dir)
        return 
    dtype = bbpp.get_dtypes(skeleton="KINECT")
    
    for i, fn in enumerate(fn_frames):           
        ll = json.load(open(fn))
        if i == 0:
            scene = [{'id':obj['id'], 'data':np.zeros(nframe, dtype=dtype)} for obj in ll['objects']]

        this_frame = get_frame(fn)
        get_skeletons(scene, ll, this_frame -1)

    return scene

def save_npy(root, file_name, arr):
    """ ex)
    root = '/home/eckim/workspace/fhe_etri/Data/BBS/Sample_BBS/BBS'
    file_name = ll.skel_fn()  #/home/eckim/workspace/fhe_etri/Data/BBS/Sample_BBS/BBS/Preds/c4/a_014_4_3.json
    arr = feature_preprocessing() 
    """
    if not os.path.isdir(root+'/npy'):
        for i in range(1, 15): # create classes dir
            os.makedirs(root+'/npy/'+str(i), exist_ok=True)
            for j in range(5):     # create scores  dir
                os.makedirs(root+'/npy/'+str(i)+'/'+str(j), exist_ok=True)

    a,b = file_name.split("/BBS/BT")
    cam, ID = a.split("/")[-2:]
    class_name, score = b.split("/")[-4:-2]
    
    if not os.path.isdir(f'{root}/npy_{cam}/{class_name}/{score}/{ID}'):
        os.makedirs(f'{root}/npy_{cam}/{class_name}/{score}/{ID}')
        
    for idx, i in enumerate(arr):
        np.save(f'{root}/npy_{cam}/{class_name}/{score}/{ID}/{cam}_{ID}_{class_name}_{score}_{idx}.npy', i)

In [22]:
root_dir = '/home/hoseung/Work/data/BBS'
# 1~53번과 54~420번 디렉토리 구조가 다름. 
fn_dirs = glob(root_dir+"/skeleton/a/*/BBS/BT/*/*/json/") + glob(root_dir+"/skeleton/a/*/BBS/BT/*/*/*/json/")

fn_dirs.sort()

In [23]:
mdtype = bbpp.get_dtypes(skeleton="COMMON")

for i, fn_dir in enumerate(fn_dirs):
    scene = load_scene(fn_dir)
    if scene is None:
        continue
    dat =[]
    for sc in scene:
        # Convert to Mobile skeleton
        marr = np.zeros(len(sc['data']), dtype=mdtype)
        kinect2mobile(sc['data'], marr)
        dat.append(marr)
    save_npy(root_dir, fn_dir, dat)

No frames found /home/hoseung/Work/data/BBS/skeleton/a/003/BBS/BT/6/3/json/
No frames found /home/hoseung/Work/data/BBS/skeleton/a/009/BBS/BT/12/4/json/
No frames found /home/hoseung/Work/data/BBS/skeleton/a/115/BBS/BT/a/10/1/json/
No frames found /home/hoseung/Work/data/BBS/skeleton/a/115/BBS/BT/a/9/1/json/
No frames found /home/hoseung/Work/data/BBS/skeleton/a/376/BBS/BT/a/10/2/json/


### 빈 디렉토리
(a, e 공통)  
003/BBS/BT/6/3/  
009/BBS/BT/12/4  
115/BBS/BT/a/9/1/  
115/BBS/BT/a/10/1/  
376/BBS/BT/a/10/2/  



In [60]:
def coco2mobile(carr, marr):
    """fill mobile_skeleton array with COCO skeleton array
       with appropriate conversions
    """
    assert len(carr) == len(marr), "coco array and mobile array are not in the same length"
    
    C2M = {"left_shoulder":"l_shoulder",
           "right_shoulder":"r_shoulder",
           "left_elbow":"l_elbow", 
           "right_elbow":"r_elbow", 
           "left_wrist":"l_hand", 
           "right_wrist":"r_hand",
           "left_hip":"l_hip", 
           "right_hip":"r_hip", 
           "left_knee":"l_knee", 
           "right_knee":"r_knee",
           "left_ankle":"l_foot", 
           "right_ankle":"r_foot"}

    # Assume neck is the mid point of shoulders
    marr['xneck'] = (carr['xleft_shoulder'] + carr['xright_shoulder'])/2
    marr['yneck'] = (carr['yleft_shoulder'] + carr['yright_shoulder'])/2
    
    # Assume (tip of) head is 2 times nose from neck
    marr['xhead'] = carr['xnose'] + (carr['xnose']-carr['xneck'])
    marr['yhead'] = carr['ynose'] + (carr['ynose']-carr['yneck'])
    
    marr['pelvis'] = 
    
    for common_field in C2M:
        marr[c2m[common_field]] = carr[common_field]

SyntaxError: invalid syntax (2686259655.py, line 28)